In [1]:
import torch
import pyaudio
import wave
import os
import AudioUtil

print(torch.__version__)
model = torch.load('data/models/V8_model_full.pth')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)
model.eval()

CHUNK = 1050
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

2.3.0+cu118
Using device: cuda:0


In [2]:
def record_audio(duration=2):
    frames = []
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    try:
        # Calculate the correct number of iterations to cover the duration
        num_frames = int((RATE / CHUNK) * duration)
        for _ in range(num_frames):
            data = stream.read(CHUNK)
            frames.append(data)
    finally:
        stream.stop_stream()
        stream.close()
        p.terminate()
    return frames

def save_temp_audio(frames, filename="temp_audio.wav"):
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(pyaudio.PyAudio().get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

def process_audio_file(filename):
    class_names = {0: 'Noise', 1: 'Knock', 2: 'kn_se'} 
    aud = AudioUtil.open(filename)
    # os.remove(filename)
    # reaud = AudioUtil.resample(aud, RATE)
    # rechan = AudioUtil.rechannel(reaud, CHANNELS)
    # dur_aud = AudioUtil.pad_trunc(rechan, 2000)
    rechan = AudioUtil.rechannel(aud, 2)
    sgram = AudioUtil.spectro_gram(rechan, n_mels=64, n_fft=1024, hop_len=None)
    # sgram = AudioUtil.spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None)
    sgram = sgram.to(device)
    model.eval()
    with torch.no_grad():
        inputs = sgram

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        outputs = model(inputs)

        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs, 1)
        # Convert predictions and actual labels to class names
        predicted_classes = [class_names.get(p.item(), p.item()) for p in prediction][0]
        print(outputs)
    # print(prediction.item())
    print(predicted_classes)
    # if prediction > 0.5:
    #     print("Klopfen erkannt!")

In [10]:
# process_audio_file for every file in train
for file in os.listdir("data/validation"):
    print(f"Processing {file}")
    process_audio_file(f"data/validation/{file}")
# process_audio_file("data/knocks/knock_10.wav")
# process_audio_file("data/noises/noise_72.wav")

Processing knock_10.wav
tensor([[9.9983e-01, 9.2165e-07, 1.6637e-04]], device='cuda:0')
Noise
Processing knock_10_var2.wav
tensor([[0.9926, 0.0012, 0.0063]], device='cuda:0')
Noise
Processing knock_10_var4.wav
tensor([[0.9922, 0.0021, 0.0057]], device='cuda:0')
Noise
Processing knock_11_var0.wav
tensor([[0.1525, 0.0432, 0.8043]], device='cuda:0')
kn_se
Processing knock_11_var1.wav
tensor([[0.7876, 0.0138, 0.1986]], device='cuda:0')
Noise
Processing knock_1_var0.wav
tensor([[1.1804e-01, 2.6795e-04, 8.8169e-01]], device='cuda:0')
kn_se
Processing knock_1_var7.wav
tensor([[2.0024e-01, 2.5707e-04, 7.9951e-01]], device='cuda:0')
kn_se
Processing knock_2_var5.wav
tensor([[9.7154e-01, 3.2900e-06, 2.8453e-02]], device='cuda:0')
Noise
Processing knock_2_var9.wav
tensor([[9.8629e-01, 5.6233e-06, 1.3704e-02]], device='cuda:0')
Noise
Processing knock_3_var6.wav
tensor([[0.0339, 0.0011, 0.9650]], device='cuda:0')
kn_se
Processing knock_3_var9.wav
tensor([[0.7319, 0.0067, 0.2614]], device='cuda:0')


In [4]:
print("Starte Klopfen-Erkennung...")
while True:
    frames = record_audio()
    save_temp_audio(frames)
    process_audio_file("temp_audio.wav")

Starte Klopfen-Erkennung...
tensor([[0., 1., 0.]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
tensor([[0., 1., 0.]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
tensor([[0., 1., 0.]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
tensor([[0., 1., 0.]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
tensor([[0., 1., 0.]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock


KeyboardInterrupt: 

In [8]:

class_names = {0: 'Noise', 1: 'Knock', 2:'Kn_se'}  # Mapping class indices to class names

# ANSI escape codes for colors
RED = '\033[91m'   # Red text
RESET = '\033[0m'  # Reset to default color

# Disable gradient updates
with torch.no_grad():
    filename = "noise_1.wav"
    aud = AudioUtil.open(filename)
    rechan = AudioUtil.rechannel(aud, 2)
    inputs = AudioUtil.spectro_gram(rechan, n_mels=64, n_fft=1024, hop_len=None)
    inputs = inputs.to(device)


    # Normalize the inputs
    inputs_m, inputs_s = inputs.mean(), inputs.std()
    inputs = (inputs - inputs_m) / inputs_s

    # Get predictions
    outputs = model(inputs.unsqueeze(0))

    # Get the predicted class with the highest score
    _, prediction = torch.max(outputs, 1)
    # Convert predictions and actual labels to class names
    predicted_classes = [class_names.get(p.item(), p.item()) for p in prediction]
    row_len = 50
    # Prepare to print 10 predictions and actuals per line
    for i in range(0, len(predicted_classes), row_len):
        pred_slice = predicted_classes[i:i+row_len]
        print('Predictions: ', ' '.join(pred_slice))

Predictions:  Knock


In [3]:
def inference(model, val_dl):
    correct_prediction = 0
    total_prediction = 0
    class_names = {0: 'Noise', 1: 'Knock', 2:'Kn_se'}  # Mapping class indices to class names

    # ANSI escape codes for colors
    RED = '\033[91m'   # Red text
    RESET = '\033[0m'  # Reset to default color

    # Disable gradient updates
    with torch.no_grad():
        for data in val_dl:
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
            outputs = model(inputs)

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs, 1)
            # Convert predictions and actual labels to class names
            predicted_classes = [class_names.get(p.item(), p.item()) for p in prediction]
            actual_classes = [class_names.get(l.item(), l.item()) for l in labels]
            row_len = 50
            # Prepare to print 10 predictions and actuals per line
            for i in range(0, len(predicted_classes), row_len):
                pred_slice = predicted_classes[i:i+row_len]
                actual_slice = actual_classes[i:i+row_len]
                pred_str = ', '.join([f'{RED if pred != actual else RESET}{pred}{RESET}' for pred, actual in zip(pred_slice, actual_slice)])
                actual_str = ', '.join([f'{RED if pred != actual else RESET}{actual}{RESET}' for pred, actual in zip(pred_slice, actual_slice)])
                print(f'Predicted: {pred_str}\nActual:    {actual_str}\n')

            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

    acc = correct_prediction / total_prediction
    print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')

In [27]:
from SoundDS import SoundDS
from torch.utils.data import DataLoader
single_ds = SoundDS('data/validation')
single_dl = DataLoader(single_ds)

inference(model, single_dl)

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Kn_se
Actual:    Knock

Predicted: Kn_se
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Kn_se
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Kn_se
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Noise
Actual:    Knock

Predicted: Kn_se
Actual:    Kn_se

Predicted: Noise
Actual:    Kn_se

Predicted: Noise
Actual:    Kn_se

Predicted: Kn_se
Actual:    Kn_se

Predicted: Kn_se
Actual:    Kn_se

Predicted: Kn_se
Act